# xmoltoppm — Hands-on demo

This notebook runs **xmoltoppm** from the command line to demonstrate its features. Each section runs a command and writes output into a folder under `demo/` in the current directory so you can inspect the results.

**Requirements:** `traj.xyz` in the project root (or adjust `-i` paths). Install with:
- `pip install -e .` (editable from repo)
- `pip install xmoltoppm[png]` for PNG output

## Setup: create output folders

Run once to create `demo/<feature>/` subfolders.

In [ ]:
import os
from pathlib import Path

for d in ["snapshot", "all_frames", "energy_overlay", "movie_preset", "styling"]:
    os.makedirs(f"demo/{d}", exist_ok=True)
print("Created demo/ subfolders.")

def show_output(path, max_frames=1):
    """Display image(s) inline: path can be a file or a folder. If folder, show first max_frames images."""
    from IPython.display import display, Image as IPImage
    try:
        from PIL import Image as PILImage
    except ImportError:
        PILImage = None
    path = Path(path)
    if path.is_file():
        files = [path]
    else:
        files = sorted(path.glob("*.ppm")) + sorted(path.glob("*.png"))
        files = files[:max_frames]
    for f in files:
        if PILImage and f.suffix.lower() == ".ppm":
            img = PILImage.open(f)
            display(img)
        else:
            display(IPImage(filename=str(f)))

## 1. Single snapshot

Render the **first frame** to one image.

**Flags:** `-i` input XYZ, `-o` output path, `-s` size (pixels), `-cc` circle style (0 flat, 1 shaded), `-il` line style (0–4).

In [ ]:
!xmoltoppm -i traj.xyz -s 500 -cc 1 -il 3 -o demo/snapshot/frame.ppm

In [ ]:
show_output("demo/snapshot")

## 2. All frames (movie)

Render **every frame** to numbered files for video encoding.

**Flags:** `-af 0` all frames (0 = skip none), `-ft 0` filenames like 0001.ppm, `-o` output directory base. Use `-mf N` to limit to first N frames for a quick demo.

In [ ]:
!xmoltoppm -i traj.xyz -s 500 -cc 1 -il 3 -af 0 -ft 0 -o demo/all_frames/ -mf 5

In [ ]:
show_output("demo/all_frames", max_frames=4)

## 3. Energy overlay from XYZ (mode 1)

**Side-by-side layout:** molecule on the left, energy vs iteration plot on the right (from the XYZ comment line). The overlay never covers the molecule.

**Flags:** `-gfe` use energy from XYZ comment, `-af 0` all frames. Writes `frame.energy.txt` next to outputs. `-mf 5` limits to 5 frames for a quick demo.

In [ ]:
!xmoltoppm -i traj.xyz -af 0 -gfe -o demo/energy_overlay/frame.ppm -mf 5

In [ ]:
show_output("demo/energy_overlay", max_frames=4)

## 4. PNG output and movie preset

**Flags:** `--png` for PNG instead of PPM; `--preset movie` for size 800, shaded circles, line style 4, shadow; `--write-run-json` records input path, options, and version for reproducibility.

In [ ]:
!xmoltoppm -i traj.xyz --preset movie -af 0 -ft 0 -o demo/movie_preset/ --png --write-run-json -mf 3

In [ ]:
show_output("demo/movie_preset", max_frames=4)

## 5. Styling: circle and line styles

**Flags:** `-cc 0` flat / `-cc 1` shaded circles; `-il` 0–4 (e.g. 4 = two-colour + directional shadow); `-sh` shadow strength (0–1).

In [ ]:
!xmoltoppm -i traj.xyz -s 400 -cc 1 -il 4 -sh 0.6 -o demo/styling/frame.ppm

In [ ]:
show_output("demo/styling")

## Help: full option list

Run `xmoltoppm -h` to print all options and flags.

In [ ]:
!xmoltoppm -h